[View in Colaboratory](https://colab.research.google.com/github/janilles/dfdapp/blob/master/dfd_risk_levels.ipynb)

# Drink Free Days app 
# RISK LEVELS ANALYSIS
Looking at the reduction in alcohol units consuption and subsquent risk levels reduction among Drink Free Days app users.


## App user definition
We're looking at users who have joined in the first 3 days of the campaign and were last seen using the app on or after day 7 of the campaign. Campaign launch date was 10 September 2018.

## Questions answered in this notebook

- **How many users have reduced their risk level** (and/or alcohol unit consumption) and by how much - before and after using the app?
  - 'Before' values are taken from the user's typical week, i.e. what they told us in the app about how much they drink in a typical week.
  - 'After' values are typical week units minus units on drink free days achieved in any or all four weeks after the campaign lauch. (For example, when a user confirms in the app that they had a drink free day on Monday in a given week then for that week units consumed are typical week values but with Monday's units removed from it.)
  - It may be the case that users record a drink free day on a day they do not drink anyway in a typicl week but that does not impact this analysis.
- **Shifts in higher/increasing/lower risk levels** before using the app (i.e. from a typical week) and after using the app (i.e. any or all selected weeks after campaign launch).
  - When a user stopped using the app in the four week period we're looking at then they are assumed to have gone back to their typical week behaviour. 
- **How many users in our target demographic** of 39-60 year olds have reduced their alcohol units consuption?

## Further analysis
Are there any patterns in behaviour or user clusters?

## Data set
Relational database behind the app - RDS instance on AWS.

# Credentials to run the notebook

## Google Drive authentication (optional)
NOTE: If login credentials are hardcoded into the database connection (code cell below) this step in not necesary. Otherwise: 

Install and authenticate [PyDrive](https://pythonhosted.org/PyDrive/index.html) for loading files from Google Drive so that database passwords aren't hardcoded into the notebook.

In [0]:
# added -q for suppressing output
!pip install -U -q PyDrive

# see PyDrive documentation for libraries code snippets
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Database connection
- Connecting to AWS RDS database with [PyMySQL](https://pymysql.readthedocs.io/en/latest/user/examples.html).
- Retruning MySQL queries as Pandas dataframes with [```read_sql()``` ](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html) function.

In [0]:
# added -q for suppressing output
!pip install -q -U pymysql

import pymysql
import pandas as pd

In [0]:
# comment out the other user/options when running this cell as necessary

# Jan's file - 'id' is Google Drive file ID
usernm_file = drive.CreateFile({'id': '1l0NedyVzKKhPJ1-_cOqF1VRt_oQyr8OL'})
passwd_file = drive.CreateFile({'id': '1YnGugBHvqjJk0nbTqN-683Agb0vaZKHo'}) 

# this variable is used in the connect function below
user_name = usernm_file.GetContentString()
user_passwd = passwd_file.GetContentString()

# If you're not using Google Drive file but are hardcoding the password
# user_passwd = password as a string

In [0]:
def connect():

    return pymysql.connect(
        
        host = "df-phereplica3.crqbvr0pveqx.eu-west-1.rds.amazonaws.com",
           
        user = user_name, # assigned in the cell above

        passwd = user_passwd, # assigned in the cell above
   
        db = "daysoff",
        
        autocommit=True

        )

connection = connect()

def sql_to_df(sql):
    """
    Returns MySQL queries as Pandas dataframes.
    """
    return pd.read_sql(sql, con = connection)

# Database tables used (optional)
Overview of avaliable data and tables used in the MySQL queries below. 
See [MySQL documentation](https://dev.mysql.com/doc/refman/8.0/en/introduction.html) for MySQL syntax.

In [0]:
# formatting column width of Pandas dataframes
# increase column width so that longer comments don't get truncated

pd.set_option('max_colwidth',100)

### App users table

In [0]:
# run pd.set_option('max_colwidth',100) if comments column gets truncated

sql_to_df("""
        SELECT
            table_name, column_name, data_type, column_comment
        FROM
            information_schema.columns
        WHERE
            table_name = 'g_appusers'
        """)

### Drinks table

In [0]:
# run pd.set_option('max_colwidth',100) if comments column gets truncated

sql_to_df("""
        SELECT 
            table_name, column_name, data_type, column_comment
        FROM
            information_schema.columns
        WHERE
            table_name = 'g_appdrinks'
        """)

### Days off table

In [0]:
# run pd.set_option('max_colwidth',100) if comments column gets truncated

sql_to_df("""
        SELECT
            table_name, column_name, data_type, column_comment
        FROM
            information_schema.columns
        WHERE
            table_name = 'g_appdaysoff'
        """)

# Report generation
- Write MySQL queries as long strings inside ```sql_to_df()``` function.  See [MySQL documentation](https://dev.mysql.com/doc/refman/8.0/en/introduction.html) for MySQL syntax reference.
- ```sql_to_df()``` returns Pandas dataframes.

### Users by day
Filtering of users (i.e. WHERE condition) explained:
- ```gender LIKE '%ale'``` is data cleaning to filter out empty values since gender is necesary for calculating risk levels
- age range: 18 (the minimum the app accepts) seems to have too many fake values and the upper cuts off outliers
- ```joined >= '2018-09-10'``` to only include app users who joined on or after the campaign start date
- ```lastseen >= '2018-09-16'``` to only include app users who have recoded some activity on (or after) seven days from the campaign launch (10 Sep 2018) so we can assume that they are using the app

**NB:** MySQL week starts on Sunday by default so: 
- ```week(lastseen)```  counts weeks from Sundays
- ```week(lastseen, 1)``` counts weeks from Monday

In [0]:
# Alcohol units calculation = (percent * ml)/ 1000

usersByDay = sql_to_df("""
        SELECT 
            U.id,
            gender,
            age,
            WEEK(lastseen, 1) AS last_week, -- MySQL week begins with Sunday by default
            day AS typical_day, -- days with no drinks in them are not recorded in database
            SUM((percent * ml)/ 1000) AS typical_units
        FROM
            g_appusers U
        LEFT JOIN g_appdrinks D
            ON U.id=D.id            
        WHERE
            gender LIKE '%ale' -- to exclude empty values
            AND
            age BETWEEN 19 AND 79 -- 18 has too many fake values, 80+ are outliers
            AND
            joined >= '2018-09-10' -- campaign start date
            AND
            lastseen >= '2018-09-16' -- at least a week from campaign launch (Monday-Sunday)
            AND
            day IS NOT NULL -- data cleaning, there were some users 'None' typical days
        GROUP BY 
            U.id, typical_day -- to sum up multiple drinks and get total units for each day
        """)

usersByDay.head()

### Users by week

In [0]:
# sum up units for each user on a typical week
usersByWeek = usersByDay.groupby(['id', 
                                  'gender', 
                                  'age', 
                                  'last_week'], as_index=False).sum()

usersByWeek.head()

In [0]:
usersByWeek.plot.scatter(2,4);

#### Exploring that line of users just above 60 units
Turns out they are drinking a bottle of wine every day.

In [0]:
import altair as alt

# increase max number of rows limit from default 5000
from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=15000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom');

In [0]:
alt.Chart(usersByWeek[usersByWeek['typical_units'] > 15]).mark_point().encode(
    x='age',
    y='typical_units',
    color='gender',
    tooltip=['id', 'typical_units', 'age']
).properties(width=600).interactive()

In [0]:
c

In [0]:
sql_to_df("""select * from g_appdrinks where id like 'ffa0206707284569'""")

In [0]:
usersByWeek[usersByWeek['typical_units'] ==63].groupby('gender')['id'].count()

### Risk in a typical week

- Defining a ```riskCalculator``` function (see docstring in code cell below for details).  
- Applying ```riskCalculator``` to every row of dataframe

#### Function definition for risk calculation

In [0]:
def riskCalculator(row, units_col):
    
    """Evaluate weekly units and gender and return risk level."""

    if row[units_col] < 15: 
        return 1 # lower

    elif ((row['gender'] == 'Male') & 
          (15 <= row[units_col] < 50)):
        return 2 # increasing

    elif ((row['gender'] == 'Female') & 
          (15 <= row[units_col] < 35)):
        return 2 # increasing

    elif ((row['gender'] == 'Male') & 
          (row[units_col] >= 50)):
        return 3 # higher

    elif ((row['gender'] == 'Female') & 
          (row[units_col] >= 35)):
        return 3 # higher

    else:
        raise ValueError(
            'Some row doesn\'t meet any of the if-elif conditions in riskCalculator.'
            ) # error message also returns row index number

### Applying risk in a typical week calculation 

In [0]:
# create a new column with risk level based on typical week units
usersByWeek['typical_risk'] = usersByWeek.apply(lambda row: 
                                                riskCalculator(row, 'typical_units'),
                                                axis=1)

usersByWeek.head()

In [0]:
alt.Chart(usersByWeek).mark_point().encode(
    x='age',
    y='typical_units',
    color=alt.Color('typical_risk',
                    scale=alt.Scale(
                    domain=['3', '2', '1'],
                    scheme='viridis'))
).properties(width=600).interactive()

### Drink free days recorded by users
Selecting form the 'days off' table which holds the drink free days recorded by users. Only positive answers to "Did you have a drink free day yesterday?" are recorded in this table.

**NB:** MySQL week starts on Sunday by default so: 
- ```week(date)```  counts weeks from Sundays

- ```week(date, 1)``` counts weeks from Monday

In [0]:
# week numbers are used in subsequent calculations so
# we need to make sure weeks start on Monday i.e. week(date, 1) 
# not Sunday which is MySQL default i.e. week(date)
# same WHERE conditions for users's table as in the users selsection

drinkFreeDays = sql_to_df("""
        SELECT 
            O.id,  
            DAYNAME(date) AS drink_free_day, 
            WEEK(date, 1) AS week_number -- week of drink free day
        FROM 
            g_appdaysoff O
        JOIN g_appusers U
            ON O.id=U.id            
        WHERE
            date >= '2018-09-10' -- campaign start date, days off table
            AND
            gender LIKE '%ale' -- to exclude empty values
            AND
            age BETWEEN 19 AND 79 -- 18 has too many fake values, 80+ are outliers
            AND
            joined >= '2018-09-10' -- campaign start date
            AND
            lastseen >= '2018-09-16' -- at least a week from campaign launch (Monday-Sunday)
        ORDER BY
            O.id, week_number
        """)

drinkFreeDays.head(10)

### Matching drink free days recorded with their corresponding alcohol units for each user

In [0]:
# grouping drink free days achieved with their typical days (typical_units)
# to calculate how many units have to be deducted from a typical week
# for each drink free day recorded for each user in each week

unitsDeductions = pd.merge(usersByDay, drinkFreeDays, 
                           # inner join to exclude drink free days achieved
                           # on a day a user doesn't drink in a typical week anyway
                           # usersByDay only includes typical days with drinks
                           how='inner',  
                           left_on=['id', 'typical_day'],
                           right_on=['id', 'drink_free_day'])

# after the merge values column 'typical_units'
# are the units to deduct for that day
# hence column renaming
unitsDeductions.rename(columns={"typical_units": "units_to_deduct"}, inplace=True)

unitsDeductions.sort_values('id').head(10)

### Units reductions by week for each user

In [0]:
unitsReductionByWeek = unitsDeductions.groupby(['id',
                                                'gender',
                                                'age',
                                                'last_week',
                                                'week_number'], 
                                                as_index=False)['units_to_deduct'].sum()

unitsReductionByWeek.sort_values('id').head(10)

In [0]:
# adding back the total typical units per week and typical risk column
unitsReductionByWeek = pd.merge(unitsReductionByWeek, usersByWeek, 
                                how='right',
                                left_on=['id', 'gender', 'age', 'last_week'],
                                right_on=['id', 'gender', 'age', 'last_week'])

unitsReductionByWeek.head(10)

### Units reduction for each week for each user

#### Function definition for weekly reductions 

In [0]:
def reductions(df=unitsReductionByWeek):
    
    """Create a column with unit reductions for each week in range()."""
    
    for i in range(37,43):
        col_name = f"reduction_{i}"
        df.loc[df['week_number'] == i, col_name] = df['units_to_deduct']
        df.fillna(0, inplace=True)    
    return df

### Reductions for each week for each user

In [0]:
result = reductions()

result.head(10)

In [0]:
# groupby dataframe by user
result = result.groupby(['id',
                         'gender',
                         'age',
                         'last_week',
                         'typical_units', 
                         'typical_risk'], 
                         as_index=False).sum()

# drop the redundant column with week numbers
result.drop(columns='week_number', inplace=True)

# rename aggregated column for clarity
result.rename(columns={'units_to_deduct':'total_reduction'}, 
              inplace=True)

result.head(10)

### Units in each week for each user

**NB: ** Users who were last seen in e.g. the first week of the campaign will get typical week values for subsequent weeks. We thereby assume they went back to their typical weeks.

#### Function definition for weekly units calculation

In [0]:
def units_for_weeks(df=result):
   
    """Create a column with units consumed for each week in range()."""
    
    for i in range(37, 43):
        col_name = f"week_{i}"
        df[col_name] = (df['typical_units'] - df[f"reduction_{i}"])
        
    return df

In [0]:
result = units_for_weeks()

result.head(10)

In [0]:
# user count by last week seen since launch 
result.groupby('last_week')['id'].count()

### Risk for each week for each user

In [0]:
risk_cols = []
week_cols = []

for i in range(37,43):
    risk_cols.append(f"risk_{i}")
    week_cols.append(f"week_{i}")
    dictionary = dict(zip(risk_cols, week_cols))

    for risk, week in dictionary.items():
        result[risk] = result.apply(lambda row: 
                                    riskCalculator(row, week), 
                                    axis=1)

result.head(10)

In [0]:
result.describe()

### Export result to CSV

In [0]:
# from google.colab import files

# result.to_csv('dfd_risk_levels.csv')
# files.download('dfd_risk_levels.csv')

# Plotting typical week

### Units consuption by age and gender (typical week)

In [0]:
uni = result.groupby(['age', 'gender'], as_index=False)['typical_units'].mean()
use = result.groupby(['age', 'gender'], as_index=False)['id'].count().rename(columns={'id':'num_of_users'})

scat = uni.merge(use, on=['age', 'gender'])

scat.head()

In [0]:
import altair as alt

alt.Chart(scat, 
          title='Average units consuption in typical week').mark_circle().encode(
          alt.X('age', scale=alt.Scale(zero=False)),
          alt.Y('typical_units', 
                title='avg. units consumed on a typical week',
                scale=alt.Scale(zero=False, padding=1)),
          color='gender',
          size=alt.Size('num_of_users', legend=alt.Legend(title="number of users")) 
          # size='num_of_users' # if you don't need to change legent title
).properties(width=500) # added width to stretch the chart

### Units consumption by age  - box plot (typical week)

In [0]:
result.iloc[:,[2,4]].head()

In [0]:
result.iloc[:,[2,4]].info()

In [0]:
# increase max number of rows limit from default 5000
from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=15000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

# Define aggregate fields
lower_box = 'q1(typical_units):Q'
lower_whisker = 'min(typical_units):Q'
upper_box = 'q3(typical_units):Q'
upper_whisker = 'max(typical_units):Q'

# Compose each layer individually
lower_plot = alt.Chart(result.iloc[:,[2,4]]).mark_rule().encode(
    y=alt.Y(lower_whisker, axis=alt.Axis(title="population")),
    y2=lower_box,
    x='age:O'
)

middle_plot = alt.Chart(result.iloc[:,[2,4]]).mark_bar(size=5.0).encode(
    y=lower_box,
    y2=upper_box,
    x='age:O'
)

upper_plot = alt.Chart(result.iloc[:,[2,4]]).mark_rule().encode(
    y=upper_whisker,
    y2=upper_box,
    x='age:O'
)

middle_tick = alt.Chart(result.iloc[:,[2,4]]).mark_tick(
    color='white',
    size=5.0
).encode(
    y='median(typical_units):Q',
    x='age:O',
)

lower_plot + middle_plot + upper_plot + middle_tick

# Plotting results

### Units consuption by age (typical week v week 37)

### Corerlation map

In [0]:
# not a age-units correlation 
# but whatt if we bin the age groups?
scat.corr()

### Units shifts week on week

# Slicing the results

## Risk level shifts over weeks
Users who stopped using the app earlier than this period are assumed to have returned to their typical weeks behaviour. 

In [0]:
riskShifts = result.groupby(['typical_risk']+risk_cols, as_index=False)['id'].count()

riskShifts.head()

In [0]:
higherSum = riskShifts[riskShifts['typical_risk'] == 3]['id'].sum()
increasingSum = riskShifts[riskShifts['typical_risk'] == 2]['id'].sum()

In [0]:
def shifts(row):
    
    if row['typical_risk'] == 3:
        return (row['id'] / higherSum *100)
        
    elif row['typical_risk'] == 2:
        return (row['id'] / increasingSum *100)
        
    else:
        return 100

In [0]:
riskShifts['% of category'] = round((riskShifts.apply(lambda row: 
                                                      shifts(row), 
                                                      axis=1)), 1)

riskShifts.sort_values('% of category', ascending=False).head(10)

In [0]:
riskShifts.iloc[1:,0:7].corr()

## 39-60

In [0]:
df3960 = result[(result['age'] > 38) & (result['age'] <= 60)]

len(df3960)

In [0]:
 any3960 = df3960[(df3960['reduction_37'] > 0) | 
                  (df3960['reduction_38'] > 0) | 
                  (df3960['reduction_39'] > 0) |
                  (df3960['reduction_40'] > 0) |
                  (df3960['reduction_41'] > 0) |
                  (df3960['reduction_42'] > 0)]
    
len(any3960)

In [0]:
# percentage of those reducing their units consuption
round(len(any3960) / (len(df3960)/100), 1)

In [0]:
df3960.groupby('typical_risk')['id'].count() 

In [0]:
df3960Shifts = df3960.groupby(['typical_risk']+risk_cols, as_index=False)['id'].count() 

df3960Shifts.head()

In [0]:
df3960Shifts['% of category'] = round((df3960Shifts.apply(lambda row: 
                                                          shifts(row), 
                                                          axis=1)), 1)

df3960Shifts.sort_values('% of category', ascending=False).head(10)